In [1]:
import asyncio
import json
import os
import uuid
import websockets
import time
from fastapi import FastAPI, WebSocket, UploadFile, File, Form, HTTPException, BackgroundTasks, WebSocketDisconnect
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
import logging
from openai import OpenAI
from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.request import CommonRequest
import aiofiles
import wave
from dotenv import load_dotenv


load_dotenv()

# Configuration - Load from environment variables in production
ALIBABA_ACCESS_KEY_ID = os.getenv("ALIBABA_ACCESS_KEY_ID")
ALIBABA_ACCESS_KEY_SECRET = os.getenv("ALIBABA_ACCESS_KEY_SECRET")
ALIBABA_REGION = os.getenv("ALIBABA_REGION", "ap-southeast-1")
ALIBABA_APPKEY = os.getenv("ALIBABA_APPKEY")
QWEN_API_KEY = os.getenv("DASHSCOPE_API_KEY")
SENTIMENT_CHECK_INTERVAL = 5  # Check sentiment every 5 seconds
SENTIMENT_THRESHOLD = 0.3  # Threshold for negative sentiment alerts (0-1)
TEMP_AUDIO_DIR = "temp_audio"

In [12]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
def get_alibaba_token():
    """Obtain an access token for Alibaba Speech Recognition service"""
    client = AcsClient(
        ALIBABA_ACCESS_KEY_ID,
        ALIBABA_ACCESS_KEY_SECRET,
        ALIBABA_REGION
    )
    
    request = CommonRequest()
    request.set_method('POST')
    request.set_domain(f'nlsmeta.{ALIBABA_REGION}.aliyuncs.com')
    request.set_version('2019-07-17')
    request.set_action_name('CreateToken')
    
    try:
        response = client.do_action_with_exception(request)
        decoded_response = response.decode("utf-8")
        logger.info(f"Raw response from Alibaba: {decoded_response}")
        response_json = json.loads(decoded_response)

        token = response_json.get("Token", {}).get("Id")
        if not token:
            logger.error("Token not found in response JSON: %s", response_json)
            raise HTTPException(status_code=500, detail="Token missing in response.")

        return token
    except Exception as e:
        logger.error(f"Failed to get Alibaba token: {e}")
        raise HTTPException(status_code=500, detail="Failed to get access token")


In [15]:
alibaba_token = get_alibaba_token()
print(f"Alibaba Token: {alibaba_token}")

INFO:__main__:Raw response from Alibaba: {"ErrMsg":"","Token":{"UserId":"5020438697749531","Id":"c021d3977ef0478dbd1a56dd56fd9dd4","ExpireTime":1747884035}}


Alibaba Token: c021d3977ef0478dbd1a56dd56fd9dd4
